Empezamos haciendo los imports necesarios y definiendo el dataframe

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("diabetes_prediction_dataset.csv")

ModuleNotFoundError: No module named 'pandas'

Transformamos los valores redundantes y los combinamos para una mayor claridad

In [ ]:
def agruparValoresRedundatesFumadores(valor):
    valor = valor.lower()
    if valor in ['never']:
        return 'never'
    elif valor in ['former', 'not current', 'ever']:
        return 'not_current'
    elif valor in ['current']:
        return 'current'
    else:
        return 'unknown'

df['smoking_history'] = df['smoking_history'].apply(agruparValoresRedundatesFumadores)

df['smoking_history'].value_counts()

Quitamos las muestras de genero "other" ya que en cantidad son irrelevantes

In [ ]:
print((df['gender'] == 'Other').sum())

df = df[df['gender'] != 'Other']

df['gender'].value_counts()


Hacemos OneHot encoding de las columnas gender y smoking_history. Además, convertimos los valores binarios True y False en 1 y 0 (no necesario)

In [ ]:
dfOneHot = pd.get_dummies(df, columns=["gender", "smoking_history"])

for column in dfOneHot.columns:
    if dfOneHot[column].dtype == bool:
        dfOneHot[column] = dfOneHot[column].astype(int)

dfOneHot.head()

Obtenemos las correlaciones entre las columnas de nuestro df, pero como realmente solo nos interesa como actuan en relacion a nuestra columna objetivo (diabetes) nos concentramos en ella

In [ ]:
correlaciones = dfOneHot.corr(method='pearson')

correlacionesConDiabetes = correlaciones['diabetes'].sort_values(key=abs, ascending=False)

print(correlacionesConDiabetes)

Usamos mathplot para ver los Boxplot de las columnas no binarias, asi podremos identificar los outliers y decidir la normalizacion

In [ ]:
columnasParaVer = ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level']

plt.figure(figsize=(15, 10))

for i, columna in enumerate(columnasParaVer):
    plt.subplot(2, 2, i + 1)
    plt.boxplot(dfOneHot[columna].dropna())
    plt.title(f'Boxplot de {columna}')
    plt.ylabel(f'Valores de {columna}')

plt.tight_layout()
plt.show()

Para el normalizado usamos RobustScaler (explicado en el informe), pero lo haremos "a mano" en lugar de con sklearn:
(Xi-Xmediana) / Xiqr

In [ ]:
dfNormalizado = dfOneHot.copy()

columnasEstandarizables = ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level']

for columna in columnasEstandarizables:
    mediana = dfNormalizado[columna].median()
    Q1 = dfNormalizado[columna].quantile(0.25)
    Q3 = dfNormalizado[columna].quantile(0.75)
    IQR = Q3 - Q1
    # Evitar división por cero
    if IQR == 0:
        dfNormalizado[columna] = 0
    else:
        dfNormalizado[columna] = (dfNormalizado[columna] - mediana) / IQR

print(dfNormalizado[columnasEstandarizables].head())


Ya con todo terminado, hacemos el nuevo csv con los datos depurados

In [ ]:
dfNormalizado.to_csv('diabetes_prediccion_normalizado.csv',index=False)